In [ ]:
import pandas as pd

## Load Excel file with mails. Inbox and Sent_items as two separate df

In [24]:
inbox_df = pd.read_excel(
    r"", sheet_name="Inbox", na_values="blank")
inbox_df = inbox_df.drop(
    inbox_df.columns[[-1, -1, -2, -3, -4, -5, -6]], axis=1)
sent_df = pd.read_excel(
    r'', sheet_name="Sent Items")
sent_df = sent_df.drop(sent_df.columns[[-1, -2, -3, -4, -5, -6]], axis=1)

In [ ]:
sent_df.head()

## Load Excel file with team members and their category for specific mailbox

In [26]:
ExcelWithTeam = pd.read_excel(r'')


In [ ]:
ExcelWithTeam

In [ ]:
index_dict = dict()
for index, name in enumerate(ExcelWithTeam["Text in mail"]): 
    index_dict[index] = name
index_dict

In [29]:
def search_fullname_sent(dict_value):
    for index, name in index_dict.items():
        if name == dict_value:
            return ExcelWithTeam["Full name"][index]

## Parse mailchain by word "From" and then try in first mail find name.

In [30]:
MyCollegues = []
for Name in ExcelWithTeam["Text in mail"]:
    MyCollegues.append(Name.strip())

MyMatch = bool
NameList = [] #list that will be converted into the column in df
for massage in sent_df['Body']:
    MyMatch = False
    massage = str(massage)
    splitMassage = massage.split("From")
    for mail in splitMassage:
        if not MyMatch:
            for Name in MyCollegues:
                if mail.lower().count(Name.lower()) != 0:
                    MyMatch = True
                    NameList.append(search_fullname_sent(Name))
                    break
        else:
            break
    if not MyMatch:
        NameList.append("Unknown")
    MyMatch = False

sent_df['Full Name'] = NameList

### Body will not be needed anymore

In [31]:
sent_df = sent_df.drop(columns="Body")

### Split recipients. Duplicate each line with new recipient. 

In [32]:
sent_df2 = sent_df.assign(Recipient=sent_df['Recipient'].str.split(';')).explode('Recipient')

for recipient in sent_df2.Recipient:
    recipient2 = str(recipient).strip()
    sent_df2['Recipient'] = sent_df2['Recipient'].replace(recipient, recipient2)
    
sent_df2 = sent_df2[(sent_df2.Recipient != "")]

### Try to assigne category for Incoming mails without it

In [33]:
inbox_df.info()
# in case some categories are missing, it would be possible still to assign some mails, 
# by searching for was selected most for each ConversationID (done in Power BI)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10308 entries, 0 to 10307
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ReceivedTime  10308 non-null  datetime64[ns]
 1   Categories    6736 non-null   object        
 2   SenderName    10308 non-null  object        
 3   ID            10308 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 322.2+ KB


In [34]:
inbox_df.Categories = inbox_df.Categories.astype(str)

## Assinge full name to Inbox categories

In [ ]:
index_dict = dict()
for index, name in enumerate(ExcelWithTeam["Category"]): 
    index_dict[index] = name
index_dict

In [36]:
def search_fullname_inbox(dict_value):
    for index, name in index_dict.items(): 
        if name == dict_value:
            return ExcelWithTeam["Full name"][index]

In [ ]:
MyMatch = bool
NameList = [] #list that will be converted into the column in df
for categor in inbox_df['Categories']:
    NameList.append(search_fullname_inbox(categor))

inbox_df['Full Name'] = NameList

inbox_df.head(10)

In [ ]:
#inbox_df.groupby("Full Name").count()
print(inbox_df.groupby("Categories").count())
print(sent_df2.groupby("Full Name").count())

In [39]:
writer = pd.ExcelWriter(
    r'')

In [40]:
inbox_df.to_excel(writer, sheet_name='Inbox',index=False)
sent_df2.to_excel(writer, sheet_name='Sent Items', index=False)
writer.save()